In [17]:
import os
import asyncio
import tempfile
import subprocess
import nest_asyncio
from dotenv import load_dotenv
from openai import AsyncOpenAI
import requests
import zipfile
from pydantic import BaseModel
from typing import Dict
import speech_recognition as sr

nest_asyncio.apply()

# ==========================================================
# STEP 1: Ensure FFmpeg exists (Auto-download if needed)
# ==========================================================
def ensure_ffmpeg():
    ffmpeg_dir = os.path.join(tempfile.gettempdir(), "ffmpeg_bin")
    ffmpeg_exe = os.path.join(ffmpeg_dir, "ffmpeg.exe")

    if not os.path.exists(ffmpeg_exe):
        print("⬇️ Downloading FFmpeg portable binary...")
        os.makedirs(ffmpeg_dir, exist_ok=True)
        url = "https://www.gyan.dev/ffmpeg/builds/ffmpeg-release-essentials.zip"
        zip_path = os.path.join(ffmpeg_dir, "ffmpeg.zip")

        # Download
        r = requests.get(url, stream=True)
        with open(zip_path, "wb") as f:
            for chunk in r.iter_content(chunk_size=8192):
                f.write(chunk)

        # Extract ffmpeg.exe only
        with zipfile.ZipFile(zip_path, "r") as zip_ref:
            for name in zip_ref.namelist():
                if name.endswith("ffmpeg.exe"):
                    zip_ref.extract(name, ffmpeg_dir)
                    extracted_path = os.path.join(ffmpeg_dir, name)
                    os.rename(extracted_path, ffmpeg_exe)
                    break

        os.remove(zip_path)

    print(f"✅ FFmpeg ready at: {ffmpeg_exe}")
    return ffmpeg_exe


ffmpeg_path = ensure_ffmpeg()


def mp3_to_wav(mp3_path: str) -> str:
    """Convert MP3 → WAV using subprocess"""
    wav_path = tempfile.NamedTemporaryFile(suffix=".wav", delete=False).name
    command = [ffmpeg_path, "-y", "-i", mp3_path, "-ar", "16000", "-ac", "1", wav_path]
    subprocess.run(command, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
    return wav_path


# ==========================================================
# STEP 2: Load Google API key
# ==========================================================
load_dotenv()
google_api_key = os.getenv("GOOGLE_API_KEY")
if not google_api_key:
    raise ValueError("❌ GOOGLE_API_KEY not found in .env")


# ==========================================================
# STEP 3: Initialize Gemini (OpenAI-compatible)
# ==========================================================
base_url = "https://generativelanguage.googleapis.com/v1beta/openai/"
model_name = "gemini-2.0-flash"
client = AsyncOpenAI(api_key=google_api_key, base_url=base_url)


# ==========================================================
# STEP 4: Pydantic Model
# ==========================================================
class AudioSummary(BaseModel):
    summary: str
    key_points: list[str]


# ==========================================================
# STEP 5: AudioSummaryAgent with Bengali support
# ==========================================================
class AudioSummaryAgent:
    def __init__(self, client: AsyncOpenAI, model: str):
        self.client = client
        self.model = model

    async def transcribe_mp3(self, mp3_path: str) -> str:
        """Convert MP3 → WAV → text (Bengali support)"""
        print(f"🎵 Converting MP3 to WAV...")
        wav_path = mp3_to_wav(mp3_path)

        recognizer = sr.Recognizer()
        with sr.AudioFile(wav_path) as source:
            print("🎙️ Transcribing Bengali speech...")
            audio_data = recognizer.record(source)
            try:
                # Bengali language code: bn-IN
                text = recognizer.recognize_google(audio_data, language="bn-IN")
                print("✅ Transcription successful.")
            except sr.UnknownValueError:
                print("⚠️ Could not recognize any Bengali speech in the audio.")
                text = ""
            except sr.RequestError as e:
                print(f"❌ Google Speech API error: {e}")
                text = ""
        return text

    async def summarize_text(self, transcript: str) -> AudioSummary:
        """Summarize transcript using Gemini"""
        if not transcript.strip():
            return AudioSummary(
                summary="No speech detected in the Bengali audio.",
                key_points=["No recognizable Bengali lyrics or speech were found."],
            )

        print("🧠 Generating summary with Gemini...")
        resp = await self.client.chat.completions.create(
            model=self.model,
            messages=[
                {
                    "role": "system",
                    "content": "You are a multilingual assistant that summarizes spoken audio content, including Bengali songs or speech.",
                },
                {"role": "user", "content": transcript},
            ],
        )
        summary = resp.choices[0].message.content.strip()

        bullets = await self.client.chat.completions.create(
            model=self.model,
            messages=[
                {"role": "user", "content": f"List 3–5 concise Bengali or English bullet points summarizing this:\n{summary}"}
            ],
        )

        key_points = [
            line.strip("-• ").strip()
            for line in bullets.choices[0].message.content.split("\n")
            if line.strip()
        ]

        return AudioSummary(summary=summary, key_points=key_points)

    async def process_audio(self, mp3_path: str) -> Dict:
        transcript = await self.transcribe_mp3(mp3_path)
        result = await self.summarize_text(transcript)
        return {
            "transcript": transcript,
            "summary": result.summary,
            "key_points": result.key_points,
        }


# ==========================================================
# STEP 6: Run main
# ==========================================================
async def main():
    mp3_path = r"C:\Virtual_Env\Emon Madhur Sandhyay(PagaiWorld.com).mp3"  # your Bengali song path
    agent = AudioSummaryAgent(client, model_name)
    result = await agent.process_audio(mp3_path)

    print("\n===== TRANSCRIPT (Bengali) =====\n", result["transcript"])
    print("\n===== SUMMARY =====\n", result["summary"])
    print("\n===== KEY POINTS =====")
    for p in result["key_points"]:
        print("•", p)


asyncio.run(main())


⬇️ Downloading FFmpeg portable binary...
✅ FFmpeg ready at: C:\Users\subha\AppData\Local\Temp\ffmpeg_bin\ffmpeg.exe
🎵 Converting MP3 to WAV...
🎙️ Transcribing Bengali speech...
✅ Transcription successful.
🧠 Generating summary with Gemini...

===== TRANSCRIPT (Bengali) =====
 এমন মধুর সন্ধ্যায় একা কি থাকা যায় এমন মধুর সন্ধ্যায় একা কি থাকা যায় খুঁজে দাও দেখি না তুমি সারাদিন আধারে প্রেমেরি হেলাতে মন যাকে চায় না আমার মন যাকে চায় এমন মধুর গান খোদা তুমি সারাদিন আঁধারে প্রেমে মন যাকে চায় আহারে আহারে আমারে আহমেদ মন চাচ্ছে না আজকে আমি প্রিয়া কাল মনে রাখবে না আসলে কেন করলি যেখানে সবাই বুঝে না দেয় না তুমি সাথীকে আমি আঁধারে পেহেলি হেলাতে মন মন যাকে চায় আমার মন যাকে চায় এমন মধুর সন্ধ্যায় একা কি খাবে আমার প্রেমেরি হেলাতে মন যাকে চায় আমি আমার মন যাকে চাই আমি দেখেছি প্রেমেরি খেলা দেখি একদিন হারিয়ে যাবো জীবনেরই মেলাতে কি বলে তোমাকে আঁখি ইশারায় বেচলো তুমি সারাদিন আধারে প্রেমেরি খেলতে মন যাকে চাই আমার মন যাতনা এমন মধুর সন্ধ্যায় একা কি থাকা যায় খুঁজে দাও বেছে নাও তুমি সাথী তুমি আঁধারে প্র